In [ ]:
import pandas as pd
import re



# Excel 파일을 데이터프레임으로 읽기
df = pd.read_excel('최종 감성사전.xlsx')

# 감성 사전을 담을 딕셔너리 초기화
sentiment_dict = {}

# 데이터프레임의 각 행을 반복하며 처리
for index, row in df.iterrows():
    word = row['Word']
    polarity = int(row['Sentiment'])
    sentiment_dict[word] = polarity

# sentiment_dict 확인
print(sentiment_dict)

In [ ]:
import pandas as pd
from konlpy.tag import Okt
import re

# Load the data
data_df = pd.read_excel("크롤링_3년_장단점/filtered_잡플래닛 리뷰 총정리_(주)유니테스트.xlsx")

# Create an instance of Okt
okt = Okt()

# Functions for tokenization and sentiment calculation
def tokenize(text):
    text = re.sub('[^\w\s]', '', text)
    return okt.morphs(text)

def calculate_sentiment_score(sentence, sentiment_dict):
    score = 0
    words = tokenize(sentence)
    for word in words:
        score += sentiment_dict.get(word, 0)
    return score

# Load sentiment dictionary
# Assuming sentiment_dict is already created from SentiWord_info.json

# # Calculate sentiment scores for each row in the dataframe
data_df['장점 점수'] = data_df['장점'].apply(lambda x: calculate_sentiment_score(str(x), sentiment_dict))
data_df['단점 점수'] = data_df['단점'].apply(lambda x: calculate_sentiment_score(str(x), sentiment_dict))

# # Display or save the results
#print(data_df)
# # Optionally, save the results to a new file
data_df.to_excel("정민장단점_점수화.xlsx")

# 모든 파일형태소 분석 

In [1]:
import pandas as pd
import os
from konlpy.tag import Okt

# 형태소 분석 함수
def analyze_text(text, stopwords):
    okt = Okt()
    morphs = okt.pos(text, stem=True)
    return [morph[0] for morph in morphs if morph[1] not in ['Alpha', 'Number', 'Punctuation', 'Foreign', 'KoreanParticle', 'Josa'] and morph[0] not in stopwords]

# 불용어 정의
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다', '되다', '것']

# 폴더 내의 모든 엑셀 파일 목록 불러오기
folder_path = './filtered'
file_list = os.listdir(folder_path)

# 각 파일에 대한 형태소 분석 수행 및 결과 저장
for file in file_list:                                                                                                           
    if file.endswith('.xlsx'):
        file_path = os.path.join(folder_path, file)
        df = pd.read_excel(file_path)

        df['장점'] = df['장점'].apply(lambda x: ' '.join(analyze_text(str(x), stopwords)))
        df['단점'] = df['단점'].apply(lambda x: ' '.join(analyze_text(str(x), stopwords)))

        # 결과를 새로운 엑셀 파일로 저장
        processed_file_path = os.path.join(folder_path, f'processed_{file}')
        df.to_excel(processed_file_path, index=False)

        print(f"File processed and saved: {processed_file_path}")

File processed and saved: ./filtered\processed_filtered_(주)에코프로에이치엔.xlsx
File processed and saved: ./filtered\processed_filtered_(주)에프알텍.xlsx
File processed and saved: ./filtered\processed_filtered_(주)에프앤가이드.xlsx
File processed and saved: ./filtered\processed_filtered_(주)엑셈.xlsx
File processed and saved: ./filtered\processed_filtered_(주)엑스게이트.xlsx
File processed and saved: ./filtered\processed_filtered_(주)엑스페릭스.xlsx
File processed and saved: ./filtered\processed_filtered_(주)엑시콘.xlsx
File processed and saved: ./filtered\processed_filtered_(주)엔바이오니아.xlsx
File processed and saved: ./filtered\processed_filtered_(주)엔에이치엔벅스.xlsx
File processed and saved: ./filtered\processed_filtered_(주)엘지헬로비전.xlsx
File processed and saved: ./filtered\processed_filtered_(주)엠디에스테크.xlsx
File processed and saved: ./filtered\processed_filtered_(주)엠씨넥스.xlsx
File processed and saved: ./filtered\processed_filtered_(주)엠에스오토텍.xlsx
File processed and saved: ./filtered\processed_filtered_(주)엠젠솔루션.xlsx
File processed an

# 점수 정규화 
- 문장 길이 영향을 최소화 하기 위해 평균 단어 수 기준 점수 정규화

1. 각 리뷰 단어수 계산
2. 모든 리뷰 대한 단어 평균 계산 
3. 단어당 점수 = 각 리뷰 점수 / 해당 리뷰 단어수
4. 점수 조정 = 단어당점수 * 평균 단어 수 

In [16]:
import os
import pandas as pd
from konlpy.tag import Okt
import re

# 폴더 경로 설정
folder_path = './filtered'  # 실제 폴더 경로로 대체하세요.

# 감성 사전 로드
df_sa = pd.read_excel('./최종/최종 감성사전.xlsx')
sentiment_dict = {row['Word']: int(row['Sentiment']) for index, row in df_sa.iterrows()}

# Okt 인스턴스 생성
okt = Okt()

# 토큰화 및 점수 계산 함수
def tokenize(text):
    text = re.sub('[^\w\s]', '', text)
    return okt.morphs(text)

def calculate_sentiment_score(sentence, sentiment_dict):
    score = 0
    words = tokenize(sentence)
    for word in words:
        score += sentiment_dict.get(word, 0)
    return score

# 지정된 폴더 내의 모든 Excel 파일 처리
combined_dfs = []

for file in os.listdir(folder_path):
    if file.endswith('.xlsx'):
        file_path = os.path.join(folder_path, file)

        # 데이터 로드
        data_df = pd.read_excel(file_path)

        # 장점과 단점에 대한 점수 계산
        data_df['장점 점수'] = data_df['장점'].apply(lambda x: calculate_sentiment_score(str(x), sentiment_dict))
        data_df['단점 점수'] = data_df['단점'].apply(lambda x: calculate_sentiment_score(str(x), sentiment_dict))

        # '단점' 열의 NaN 값을 빈 문자열("")로 대체합니다.
        data_df['단점'] = data_df['단점'].fillna("")

        # 각 리뷰의 단어 수를 계산합니다.
        # 이 예에서는 '장점'과 '단점' 열의 단어 수를 각각 계산하고 합산합니다.
        data_df['장점_단어_수'] = data_df['장점'].astype(str).str.split().str.len()
        data_df['단점_단어_수'] = data_df['단점'].astype(str).str.split().str.len()
        data_df['총_단어_수'] = data_df['장점_단어_수'] + data_df['단점_단어_수']

        # 모든 리뷰에 대한 단어 수의 평균을 계산합니다.
        average_word_count = data_df['총_단어_수'].mean()

        # 각 리뷰 점수를 해당 리뷰의 단어 수로 나누어 '단어당 점수'를 계산합니다.
        data_df['장점_단어당_점수'] = data_df['장점 점수'] / data_df['장점_단어_수']
        data_df['단점_단어당_점수'] = data_df['단점 점수'] / data_df['단점_단어_수']

        # '단어당 점수'에 평균 단어 수를 곱하여 점수를 조정합니다.
        data_df['조정된_장점_점수'] = data_df['장점_단어당_점수'] * average_word_count
        data_df['조정된_단점_점수'] = data_df['단점_단어당_점수'] * average_word_count
        data_df['조정된_총합_점수'] = data_df['조정된_장점_점수'] + data_df['조정된_단점_점수']

        # 결과 저장
        output_file_path = os.path.join(folder_path, 'scored_' + file)
        data_df.to_excel(output_file_path, index=False)
        
        # 저장된 데이터프레임을 리스트에 추가
        combined_dfs.append(data_df)

# 모든 파일을 하나로 병합하여 최종 결과 파일로 저장
combined_df = pd.concat(combined_dfs)
combined_output_path = os.path.join(folder_path, '점수_정규화_총합.xlsx')

# 조정된 장점 점수와 단점 점수의 총합을 구합니다.
combined_df['조정된_총합_점수'] = combined_df['조정된_장점_점수'] + combined_df['조정된_단점_점수']

# 조정된 총합 점수로 정렬하여 상위 리뷰를 확인합니다.
top_reviews = combined_df.sort_values(by='조정된_총합_점수', ascending=False)

# 상위 리뷰의 데이터를 출력합니다.
print(top_reviews[['조정된_장점_점수', '조정된_단점_점수', '조정된_총합_점수']].head())

# '기업명'과 '총점수' 컬럼만 있는 엑셀 파일 생성
final_result = top_reviews[['기업명', '조정된_총합_점수']]
final_output_path = os.path.join(folder_path, '기업별_총점수.xlsx')
final_result.to_excel(final_output_path, index=False)


    조정된_장점_점수  조정된_단점_점수   조정된_총합_점수
0   98.466667  70.333333  168.800000
0   98.466667  70.333333  168.800000
20  76.643016  67.746951  144.389967
20  76.643016  67.746951  144.389967
20  76.643016  67.746951  144.389967


KeyError: "['기업명'] not in index"

In [17]:
import os
import pandas as pd

# 폴더 경로 설정
folder_path = './filtered'  # 실제 폴더 경로로 대체하세요.

# 모든 파일의 데이터를 병합한 데이터프레임 생성
combined_df = pd.DataFrame()

# 지정된 폴더 내의 모든 Excel 파일 처리
for file in os.listdir(folder_path):
    if file.endswith('.xlsx') and file.startswith('scored_scored_scored_processed_filtered_'):
        # 파일명에서 기업명 추출
        company_name = file.replace('scored_scored_scored_processed_filtered_', '').replace('.xlsx', '')
        file_path = os.path.join(folder_path, file)

        # 데이터 로드
        data_df = pd.read_excel(file_path)

        # 기업명을 데이터프레임에 추가
        data_df['기업명'] = company_name

        # 데이터프레임 병합
        combined_df = pd.concat([combined_df, data_df])

# 각 기업별로 '조정된 총합 점수'의 합을 계산
final_scores = combined_df.groupby('기업명')['조정된_총합_점수'].sum().reset_index()

# 결과를 엑셀 파일로 저장
final_output_path = os.path.join(folder_path, '기업별_총점수.xlsx')
final_scores.to_excel(final_output_path, index=False)

# 상위 5개 기업의 총점수를 출력
print(final_scores.sort_values(by='조정된_총합_점수', ascending=False).head())


            기업명      조정된_총합_점수
17        excel  664778.328788
50    아모레퍼시픽(주)   29455.863789
56           안랩   12011.146617
19         셀트리온    9356.425393
30  신세계인터내셔날(주)    8387.374136
